In [ ]:
# @title 👇 环境配置
from pathlib import Path
import urllib.request
import sys
import os
# SD Scripts Manager 核心下载地址
SD_SCRIPTS_IPYNB_CORE_URL = "https://github.com/licyk/sd-webui-all-in-one/raw/main/sd_scripts_ipynb_core.py"
FORCE_DOWNLOAD_CORE = False  # 设置为 True 时, 即使 SD Scripts Manager 已存在也会重新下载
try:
    print(f"[Repo Mirror Tools] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}") # type: ignore
except Exception as _:
    JUPYTER_ROOT_PATH = Path(os.getcwd())
    sys.path.append(str(JUPYTER_ROOT_PATH.resolve()))
    print(f"[Repo Mirror Tools] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}")
    SD_SCRIPTS_IPYNB_CORE_PATH = JUPYTER_ROOT_PATH / \
        "sd_scripts_ipynb_core.py"  # SD Scripts Manager 核心保存路径
try:
    if SD_SCRIPTS_IPYNB_CORE_PATH.exists() and not FORCE_DOWNLOAD_CORE:
        print("[Repo Mirror Tools] SD Scripts Manager 核心模块已存在")
    else:
        urllib.request.urlretrieve(
            SD_SCRIPTS_IPYNB_CORE_URL, SD_SCRIPTS_IPYNB_CORE_PATH)
        print("[Repo Mirror Tools] SD Scripts Manager 核心模块下载成功")
except Exception as e:
    raise Exception(f"SD Scripts Manager 核心模块下载错误: {e}")
from sd_scripts_ipynb_core import logger, VERSION, BaseManager
logger.info("SD Scripts Manager 版本: %s", VERSION)
logger.info("核心模块初始化完成")
#################################################################################################################


class RepoMirrorTools(BaseManager):
    """镜像制作工具"""

    def clean_package_cache(self) -> bool:
        """清理 APT, Pip, uv 的缓存

        :return `bool`: 清理成功时返回`True`
        """
        logger.info("清理缓存中")
        try:
            self.run_cmd(["apt", "clean"])
            self.run_cmd([sys.executable, "-m", "pip", "cache", "purge"])
            self.run_cmd(["uv", "cache", "clean"])
            logger.info("缓存清理完成")
            return True
        except Exception as e:
            logger.error("清理缓存时出现错误: %s", e)
            return False

    def verify_huggingface_token(self, hf_token: str) -> bool:
        """HuggingFace Token 验证

        :param hf_token `(str)`: HuggingFace 账号 Token
        :return `bool`: 验证成功时返回`True`, 否则返回`False`
        """
        from huggingface_hub import HfApi
        logger.info("验证 HuggingFace Token 中")
        api = HfApi()
        try:
            api.whoami(hf_token)
            logger.info("HuggingFace Token 验证成功")
            return True
        except Exception as e:
            logger.error("HuggingFace Token 验证失败", e)
            return False

    def verify_modelscope_token(self, ms_token: str) -> bool:
        """验证 ModelScope Token

        :param ms_token `(str)`: ModelScope 账号 Token
        :param `bool`: 验证成功时返回`True`, 否则返回`False`
        """
        from modelscope import HubApi
        api = HubApi()
        logger.info("验证 ModelScope Token 中")
        try:
            api.login(ms_token)
            logger.info("ModelScope Token 验证成功")
            return True
        except Exception as e:
            logger.error("ModelScope Token 验证失败", e)
            return False

    def install(
        self,
        use_uv: bool | None = True,
        huggingface_token: str | None = None,
        modelscope_token: str | None = None,
    ) -> None:
        logger.info("配置镜像制作工具环境中")
        self.mirror.configure_pip()
        self.env.install_manager_depend(use_uv)
        self.clean_package_cache()
        self.utils.clear_up()
        if huggingface_token is not None and not self.verify_huggingface_token(huggingface_token):
            logger.warning("请检查 HuggingFace Token 是否可用")
        if modelscope_token is not None and not self.verify_modelscope_token(modelscope_token):
            logger.warning("请检查 ModelScope Token 是否可用")
        self.restart_repo_manager(
            hf_token=huggingface_token,
            ms_token=modelscope_token,
        )
        logger.info("配置镜像制作工具环境完成")


###############################################

# @markdown #### HuggingFace Token
HF_TOKEN = ""  # @param {type:"string"}
# @markdown #### ModelScope Token
MS_TOKEN = ""  # @param {type:"string"}
WORKSPACE = "/content"

###############################################
os.chdir(WORKSPACE)
mirror_maker = RepoMirrorTools(WORKSPACE, "__MIRROR_MAKER__")
mirror_maker.install(
    use_uv=True,
    huggingface_token=HF_TOKEN or None,
    modelscope_token=MS_TOKEN or None,
)

In [ ]:
#@title 👇 上传文件夹到 ModelScope

#@markdown #### ModelScope 仓库 ID
repo_id = "" #@param {type:"string"}
#@markdown #### ModelScope 仓库类型
repo_type = "model" #@param ["model", "dataset", "space"]
#@markdown #### 当仓库不存在时自动创建的仓库的可见性
visibility = True #@param {type:"boolean"}
#@markdown #### 上传的文件夹
upload_path = "/content/model" #@param {type:"string"}

mirror_maker.repo.upload_files_to_repo(
    api_type="modelscope", # Api 类型
    repo_id=repo_id, # Modelscope 的仓库地址
    repo_type=repo_type, # ModelScope 仓库类型
    visibility=visibility, # 新建的仓库可见性
    upload_path=upload_path # 要上传的文件夹
)

In [ ]:
#@title 👇 上传文件夹到 HuggingFace

#@markdown #### HuggingFace 仓库 ID
repo_id = "" #@param {type:"string"}
#@markdown #### HuggingFace 仓库类型
repo_type = "model" #@param ["model", "dataset", "space"]
#@markdown #### 当仓库不存在时自动创建的仓库的可见性
visibility = True #@param {type:"boolean"}
#@markdown #### 上传的文件夹
upload_path = "/content/model" #@param {type:"string"}

mirror_maker.repo.upload_files_to_repo(
    api_type="huggingface", # Api 类型
    repo_id=repo_id, # HuggingFace 仓库地址
    repo_type=repo_type, # HuggingFace 仓库种类
    visibility=visibility, # 新建的仓库可见性
    upload_path=upload_path # 要上传文件的目录
)

In [ ]:
#@title 👇 从 HuggingFace 下载仓库中的单个文件

#@markdown #### HuggingFace 仓库 ID
repo_id = "" #@param {type:"string"}
#@markdown #### HuggingFace 仓库类型
repo_type = "model" #@param ["model", "dataset", "space"]
#@markdown #### 文件在仓库中的路径
filename = "" #@param {type:"string"}
#@markdown #### 下载到本地的路径
local_dir = "/content/model" #@param {type:"string"}
cache_dir_1 = os.path.join(local_dir, ".cache")
cache_dir_2 = os.path.join(local_dir, ".huggingface")

mirror_maker.repo.hf_api.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    repo_type=repo_type,
    local_dir=local_dir,
)

!rm -rf "{cache_dir_1}" "{cache_dir_2}"

In [ ]:
#@title 👇 从 HuggingFace 下载整个仓库

#@markdown #### HuggingFace 仓库 ID
repo_id = "" #@param {type:"string"}
#@markdown #### HuggingFace 仓库类型
repo_type = "model" #@param ["model", "dataset", "space"]
#@markdown #### 下载到本地的路径
local_dir = "/content" #@param {type:"string"}

mirror_maker.repo.hf_api.snapshot_download(
    repo_id=repo_id,
    repo_type=repo_type,
    local_dir=local_dir,
)

In [ ]:
#@title 👇 下载文件

#@markdown #### 下载链接
url = "" #@param {type:"string"}
#@markdown #### 下载到本地的路径
path = "/content" #@param {type:"string"}
#@markdown #### 保存的文件名
filename = "" #@param {type:"string"}
#@markdown #### 下载工具
download_tool = "aria2" #@param ["aria2", "request"]

mirror_maker.downloader.download_file(
    url=url,
    path=path,
    save_name=filename or None,
    tool=download_tool,
)

In [ ]:
#@title 👇 清理 Python 软件包
logger.info("清理 Python 软件包中")
!uv pip uninstall \
    tensorboard tensorboard-data-server tensorflow tensorflow-datasets tensorflow-estimator tensorflow-gcs-config tensorflow-hub tensorflow-io-gcs-filesystem tensorflow-metadata tensorflow-probability tensorstore \
    torch torchaudio torchsummary torchtext torchvision triton \
    opencv-python-headless opencv-python opencv-contrib-python \
    Sphinx sphinxcontrib-applehelp sphinxcontrib-devhelp sphinxcontrib-htmlhelp sphinxcontrib-jsmath sphinxcontrib-qthelp sphinxcontrib-serializinghtml
logger.info("Python 软件包清理完成")